In [2]:
import os
import json
from stelaro.data import format, ncbi
import numpy as np
import random

DATA_DIRECTORY = "../data/"
SUMMARY_DIRECTORY = DATA_DIRECTORY + "ncbi_genome_summaries/"
NCBI_TAXONOMY_DIRECTORY = DATA_DIRECTORY + "ncbi_taxonomy/"
BERTAX_DIRECTORY = DATA_DIRECTORY + "bertax/final/"
BERTAX_DATASET_DIRECTORY = BERTAX_DIRECTORY + "final_model_data_seperate_fasta_per_superkingdom/data/fass2/projects/fk_read_classification/dna_sequences/fragments/genomic_fragments_80_big/"
BERTAX_DOMAINS = (
    "Archaea_db.fa",
    "Bacteria_db.fa",
    "Eukaryota_db.fa",
    "Viruses_db.fa",
)
BERTAX_STATISTIC_DIRECTORY = BERTAX_DIRECTORY + "statistics/"
PROCESSED_PRETRAINING_DATA = DATA_DIRECTORY + "bertax/pretraining/processed/"
SEQUENCE_LENGTH = 1500
N_MINIMUM_READS_PER_TAXON = 10_000


def mkdir(path: str) -> None:
    """Create a directory if it does not exist."""
    if not os.path.exists(path):
        os.makedirs(path)


mkdir(BERTAX_STATISTIC_DIRECTORY)

# One-Shot

In [3]:
BERTAX_TRAIN = BERTAX_DIRECTORY + "train/"
BERTAX_VALIDATION = BERTAX_DIRECTORY + "validation/"
BERTAX_TEST = BERTAX_DIRECTORY + "test/"

import numpy as np
from torch.utils.data import DataLoader
import json
from torch.optim import Adam
import matplotlib.pyplot as plt
from time import time
from torch import nn, exp

from stelaro.data import format
from stelaro import models

LENGTH = 1500
BATCH_SIZE = 128


train_data = DataLoader(
    models.SyntheticTetramerDataset(BERTAX_TRAIN),
    batch_size=BATCH_SIZE,
    shuffle=True
)
validation_data = DataLoader(
    models.SyntheticTetramerDataset(BERTAX_VALIDATION),
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_data = DataLoader(
    models.SyntheticTetramerDataset(BERTAX_TEST),
    batch_size=BATCH_SIZE,
    shuffle=True
)

with open(BERTAX_DIRECTORY + "statistics/map.json", "r") as f:
    mapping = json.load(f)


def benchmark(
        classifier: models.BaseClassifier,
        name: str,
        max_epochs: int = 20,
        loss_fn = nn.CrossEntropyLoss(),
        evaluation_interval=5000,
        learning_rate: float = 0.001
    ):
    parameters = classifier.get_parameters()
    if parameters:
        optimizer = Adam(classifier.get_parameters(), lr=0.001)
        total_params = sum(param.numel() for param in parameters)
        print(f"Number of parameters: {total_params:_}")
    else:
        optimizer = None
    a = time()
    losses, f1, validation_losses = classifier.train(
        train_data,
        validation_data,
        optimizer,
        max_n_epochs=max_epochs,
        patience=3,
        loss_function=loss_fn,
        loss_function_type="supervised",
        evaluation_interval=evaluation_interval
    )
    b = time()
    print(f"Training took {(b - a):.3f} s.")
    if losses:
        fig, ax = plt.subplots(1, 2, figsize=(12, 4))
        x = list(range(len(losses)))
        ax[0].plot(x, losses, label="Training")
        ax[0].plot(x, validation_losses, label="Validation")
        ax[0].set(xlabel='Epochs', ylabel='Loss')
        ax[0].set_title("Normalized Loss Against Epochs")
        ax[0].legend()
        ax[1].set(xlabel='Epochs', ylabel="f1")
        ax[1].set_title("F1 Score")
        r = 0
        for f in f1:
            ax[1].plot(x, f, label=f'Rank {r}')
            r += 1
        ax[1].legend()
        fig.suptitle(f"Classification Training for {name}")
        plt.show()
    result = models.evaluate(classifier, test_data, "cuda", mapping)
    rounded_result = [float(f"{r:.5}") for r in result]
    print(f"F1: {rounded_result}")
    result = models.evaluate_precision(classifier, test_data, "cuda", mapping)
    rounded_result = [float(f"{r:.5}") for r in result]
    print(f"Precision: {rounded_result}")
    return classifier

In [ ]:
from torch.optim import Adam
import torch
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel, MambaConfig
from mamba_ssm import Mamba
from torch import nn

# 256, 128, 2, 16, 8, 2, 0.1, batch 128, noClip, CEL
# F1: [0.94682, 0.88288]
# Precision: [0.94012, 0.88521]

# 256, 64, 2, 16, 8, 2, 0.1, batch 128, noClip, CEL
# F1: [0.92456, 0.8333]
# Precision: [0.93198, 0.84133]

# 256, 128, 2, 16, 8, 2, 0.1, batch 64, noClip, CEL
# F1: [0.94531, 0.88431]
# Precision: [0.94236, 0.88785]

# 256, 128, 2, 16, 8, 3, 0.1, batch 128, noClip, CEL*
# F1: [0.95005, 0.88783]
# Precision: [0.94598, 0.89011]

# 256, 256, 2, 16, 8, 2, 0.1, batch 64, 5 epochs, noClip, CEL
# F1: [0.95149, 0.88721]
# Precision: [0.95105, 0.88929]

# 256, 128, 2, 16, 16, 2, 0.1, batch 128, noClip, CEL
# F1: [0.95237, 0.88463]
# Precision: [0.95195, 0.88635]

# 256, 128, 2, 16, 16, 2, 0.1, batch 128, clip, focal mean
# F1: [0.94285, 0.87642]
# Precision: [0.93603, 0.87868]

# 256, 128, 2, 16, 16, 2, 0.1, batch 128, clip, focal sum
# F1: [0.94163, 0.87185]
# Precision: [0.93829, 0.87553]

# 256, 128, 2, 16, 16, 2, 0.1, batch 128, clip, penalty sum
# F1: [0.91501, 0.68972]
# Precision: [0.90453, 0.7091]

# 256, 128, 2, 16, 8, 2, 0.1, batch 128, noClip, focal sum
# F1: [0.93204, 0.85329]
# Precision: [0.93095, 0.85767]

# 256, 128, 3, 16, 8, 2, 0.1, batch 128, noClip, CEL
# F1: [0.90227, 0.80133]
# Precision: [0.89234, 0.80588]


class MambaSequenceClassifier(nn.Module):
    def __init__(
        self,
        N: int,
        num_classes: int,
        vocab_size: int = 256,
        d_model: int = 128,
        n_layers: int = 3,
        d_state: int = 16,
        d_conv: int = 4,
        expand: int = 2,
        pooling = "mean",
        dropout: float = 0.1,
    ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.layers = nn.ModuleList([
            Mamba(d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand)
            for _ in range(n_layers)
        ])
        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()
        self.classifier = nn.Linear(d_model, num_classes)
        self.pooling = pooling

    def forward(self, x: torch.LongTensor) -> torch.Tensor:
        h = self.embedding(x).to(dtype=torch.get_default_dtype())
        # h = h.to(torch.float16)
        for block in self.layers:
            h = block(h)   # each Mamba block returns [B, L, d_model]
        h = self.norm(h)
        pooled = h.mean(dim=1)  # [B, d_model]
        pooled = self.dropout(pooled)
        logits = self.classifier(pooled)  # [B, num_classes]
        return logits


classifier = models.Classifier(
    LENGTH // 4,
    mapping,
    "cuda",
    MambaSequenceClassifier,
    format.to_tetramers,
    True
)
# classifier.model = classifier.model.to(torch.float16)

import torch.nn.functional as F
def focal_loss(inputs, targets, alpha=1, gamma=2):
    num_classes = inputs.shape[1]
    targets = F.one_hot(targets, num_classes=num_classes).float()
    bce_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
    pt = exp(-bce_loss)  # Convert BCE loss to probability
    focal_loss = alpha * (1 - pt) ** gamma * bce_loss  # Apply focal adjustment
    return focal_loss.sum()


def create_penalty_matrix(mapping):
    d = len(mapping)
    t = torch.zeros((d, d))
    n_ranks = len(mapping['0'])
    for i in mapping:
        for j in range(d):
            j = str(j)
            union_length = 0
            for a, b in zip(mapping[i], mapping[j]):
                if a == b:
                    union_length += 1
                else:
                    break
            penalty = ((n_ranks - union_length) / n_ranks) ** 0.5
            t[int(i), int(j)] = penalty
    return t
penalty_matrix = create_penalty_matrix(mapping).to("cuda")
def penalty_cross_entropy(y_pred, y_true):
    probs = F.softmax(y_pred, dim=-1)  # [B, M]
    penalty_rows = penalty_matrix[y_true]  # [B, M]
    loss = -torch.sum((1 - penalty_rows) * torch.log(probs + 1e-12), dim=-1)
    return loss.sum()
def hierarchical(input, targets):
    return models.penalized_cross_entropy(input, targets, penalty_matrix)


model = benchmark(
    classifier,
    "MAMBA",
    max_epochs=10,
    loss_fn=nn.CrossEntropyLoss()
)
matrix = models.confusion_matrix(model, test_data, "cuda", mapping)
plt.matshow(matrix)
plt.show()
np.set_printoptions(precision=3, suppress=True, linewidth=500, threshold=np.inf)
print(matrix)

Number of parameters: 388_785


 22%|██▏       | 2003/9196 [04:03<6:20:14,  3.17s/it] 

P: [0.57526, 0.22857]


 44%|████▎     | 4002/9196 [08:44<5:56:33,  4.12s/it]

P: [0.65106, 0.35171]


...

P: [0.90441, 0.80098]


100%|██████████| 9196/9196 [21:27<00:00,  7.14it/s]  


8/10 T loss: 2.00815. V loss: 2.05332. F1: [0.90019, 0.79307]. P: [0.89525, 0.79748] Patience: 3


 16%|█▌        | 1462/9196 [03:12<16:56,  7.61it/s]


KeyboardInterrupt: 

In [31]:
result = models.evaluate(classifier, test_data, "cuda", mapping)
rounded_result = [float(f"{r:.5}") for r in result]
print(f"F1: {rounded_result}")
result = models.evaluate_precision(classifier, test_data, "cuda", mapping)
rounded_result = [float(f"{r:.5}") for r in result]
print(f"Precision: {rounded_result}")

F1: [0.90227, 0.80133]
Precision: [0.89234, 0.80588]


In [12]:
print(mapping)
m = models.create_penalty_matrix(mapping)
print(m)

{'0': ['Archaea', 'Methanobacteriota'], '1': ['Archaea', 'Nitrososphaerota'], '2': ['Archaea', 'Thermoplasmatota'], '3': ['Archaea', 'Thermoproteota'], '4': ['Bacteria', 'Actinomycetota'], '5': ['Bacteria', 'Aquificota'], '6': ['Bacteria', 'Bacillota'], '7': ['Bacteria', 'Bacteroidota'], '8': ['Bacteria', 'Bdellovibrionota'], '9': ['Bacteria', 'Campylobacterota'], '10': ['Bacteria', 'Chlamydiota'], '11': ['Bacteria', 'Chlorobiota'], '12': ['Bacteria', 'Chloroflexota'], '13': ['Bacteria', 'Cyanobacteriota'], '14': ['Bacteria', 'Deinococcota'], '15': ['Bacteria', 'Fusobacteriota'], '16': ['Bacteria', 'Gemmatimonadota'], '17': ['Bacteria', 'Lentisphaerota'], '18': ['Bacteria', 'Mycoplasmatota'], '19': ['Bacteria', 'Myxococcota'], '20': ['Bacteria', 'Nitrospirota'], '21': ['Bacteria', 'Planctomycetota'], '22': ['Bacteria', 'Pseudomonadota'], '23': ['Bacteria', 'Rhodothermota'], '24': ['Bacteria', 'Spirochaetota'], '25': ['Bacteria', 'Thermodesulfobacteriota'], '26': ['Bacteria', 'Thermotog

# 1-NT Tokens

In [ ]:
from torch.optim import Adam
import torch
from mamba_ssm import Mamba
from torch import nn

# 256, 64, 2, 16, 8, 2, 0.1, batch 128
# F1: [0.88082, 0.78494]
# P: [0.89169, 0.80668]

class MambaSequenceClassifier(nn.Module):
    def __init__(
        self,
        N: int,
        num_classes: int,
        vocab_size: int = 4,
        d_model: int = 64,
        n_layers: int = 2,
        d_state: int = 16,
        d_conv: int = 8,
        expand: int = 2,
        pooling = "mean",
        dropout: float = 0.1,
    ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.layers = nn.ModuleList([
            Mamba(d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand)
            for _ in range(n_layers)
        ])
        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()
        self.classifier = nn.Linear(d_model, num_classes)
        self.pooling = pooling

    def forward(self, x: torch.LongTensor) -> torch.Tensor:
        h = self.embedding(x).to(dtype=torch.get_default_dtype())
        for block in self.layers:
            h = block(h)   # each Mamba block returns [B, L, d_model]
        h = self.norm(h)
        pooled = h.mean(dim=1)  # [B, d_model]
        pooled = self.dropout(pooled)
        logits = self.classifier(pooled)  # [B, num_classes]
        return logits


classifier = models.Classifier(LENGTH, mapping, "cuda", MambaSequenceClassifier, format.to_digits)
model = benchmark(
    classifier,
    "MAMBA",
    max_epochs=10,
)
matrix = models.confusion_matrix(model, test_data, "cuda", mapping)
plt.matshow(matrix)
plt.show()
np.set_printoptions(precision=3, suppress=True, linewidth=500, threshold=np.inf)
print(matrix)

Number of parameters: 69_873


 54%|█████▍    | 5001/9196 [28:50<8:21:51,  7.18s/it]

P: [0.87169, 0.7341]


100%|██████████| 9196/9196 [1:36:26<00:00,  1.59it/s]     


Halting evaluation after 28032 data points.
1/10 T loss: 13.99071. V loss: 10.32095. F1: [0.86647, 0.74595]. P: [0.88074, 0.77692] Patience: 3


...

P: [0.90584, 0.8145]


100%|██████████| 9196/9196 [1:57:10<00:00,  1.31it/s]   


Halting evaluation after 38144 data points.
2/10 T loss: 8.43410. V loss: 8.98794. F1: [0.88082, 0.78494]. P: [0.89169, 0.80668] Patience: 3


  1%|          | 94/9196 [01:11<1:54:40,  1.32it/s]


KeyboardInterrupt: 

# Pretraining

In [ ]:
# Obtain pretraining data
DATA_DIRECTORY = "../data/"
BERTAX_DIRECTORY = DATA_DIRECTORY + "bertax/pretraining/"
BERTAX_DATASET_DIRECTORY = BERTAX_DIRECTORY + "pretraining_dataset/data/fass2/projects/fk_read_classification/dna_sequences/fragments/genomic_fragments_80/"
BERTAX_DOMAINS = (
    "Archaea_db.fa",
    "Bacteria_db.fa",
    "Eukaryota_db.fa",
    "Viruses_db.fa",
)

total = 0
for domain in BERTAX_DOMAINS:
    with open(BERTAX_DATASET_DIRECTORY + domain, "r") as f:
        for line in f:
            if line.startswith(">"):
                total += 1
    print(f"Domain: {domain}. Total: {total}")

x = np.zeros((total, 1500 // 4), dtype=np.uint8)
mkdir(PROCESSED_PRETRAINING_DATA)
i = 0
for domain in BERTAX_DOMAINS:
    with open(BERTAX_DATASET_DIRECTORY + domain, "r") as f:
        for line in f:
            if not line.startswith(">"):
                sequence = line.strip().upper()
                characters = set(sequence)
                if len(characters) == 4:
                    encoding = format.encode_tetramer(sequence)
                    x[i] = encoding
                    i += 1
np.save(PROCESSED_PRETRAINING_DATA + "/x.npy", x)

In [9]:
from torch.optim import Adam, AdamW
import torch
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel, MambaConfig
from mamba_ssm import Mamba
from torch import nn


# 128, 2, 16, 4, 2, mean, 0.1, noClip, pretraining: 10_000 -> 0.15 MLM
# F1: [0.9365, 0.86395]
# Precision: [0.93262, 0.87061]

# 128, 2, 16, 4, 2, mean, 0.1, noClip, pretraining: 20_000 -> 0.15 MLM
# Precision: [0.93845, 0.86496] (incomplete training - no difference with 10k steps)
# Lowest MLM loss: 0.1352
# Fine-tuning:
#   Epoch 1: F1: [0.88529, 0.75644]. P: [0.89593, 0.76523]
#   Epoch 2: F1: [0.91113, 0.81615]. P: [0.91365, 0.82102]

# 128, 2, 16, 4, 2, mean, 0.1, noClip, pretraining: 100_000 -> 0.15 MLM
# 1/10 T loss: 2.97148. V loss: 2.46173. F1: [0.87928, 0.75458]. P: [0.87739, 0.77189] Patience: 3
# 2/10 T loss: 1.97898. V loss: 1.84719. F1: [0.91845, 0.82066]. P: [0.92227, 0.82479] Patience: 3
# 5/10 T loss: 1.37992. V loss: 1.43371. F1: [0.9326, 0.86184]. P: [0.92561, 0.86369]

# 128, 2, 16, 4, 2, mean, 0.1, noClip, pretraining: 50_000 new -> 0.15 MLM
# 1/10 T loss: 3.11947. V loss: 2.29345. F1: [0.89625, 0.77654]. P: [0.89769, 0.78591] Patience: 3
# 2/10 T loss: 2.04814. V loss: 1.93529. F1: [0.91427, 0.81052]. P: [0.913, 0.81998] Patience: 3
# 3/10 T loss: 1.70350. V loss: 1.71731. F1: [0.92385, 0.83688]. P: [0.9214, 0.84447] Patience: 3

# 128, 2, 16, 4, 2, mean, 0.1, noClip, pretraining: 40_000 new -> 0.15 MLM, lr = 0.0001
# 1/10 T loss: 3.12365. V loss: 2.35835. F1: [0.8859, 0.76903]. P: [0.88096, 0.77557] Patience: 3
# 2/10 T loss: 1.99888. V loss: 1.85153. F1: [0.9151, 0.8191]. P: [0.9125, 0.82259] Patience: 3
# 3/10 T loss: 1.68860. V loss: 1.62224. F1: [0.92761, 0.84333]. P: [0.92996, 0.84681] Patience: 3

# 128, 3 (norms), 16, 4, 2, mean, 0.1, clip, pretraining: 10_000 new -> 0.2 MLM, lr = 0.0001


class MambaBlock(nn.Module):
    def __init__(self, d_model, d_state, d_conv, expand):
        super().__init__()
        self.norm = nn.LayerNorm(d_model)
        self.mamba = Mamba(
            d_model=d_model,
            d_state=d_state,
            d_conv=d_conv,
            expand=expand
        )

    def forward(self, x):
        # Pre-norm + residual connection
        return x + self.mamba(self.norm(x))


class MambaSequenceClassifier(nn.Module):
    def __init__(
        self,
        N: int,
        num_classes: int,
        vocab_size: int = 256,
        d_model: int = 128,
        n_layers: int = 3,
        d_state: int = 16,
        d_conv: int = 4,
        expand: int = 2,
        pooling = "mean",
        dropout: float = 0.1,
    ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size + 1, d_model)
        self.layers = nn.ModuleList([
            # Mamba(d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand)
            MambaBlock(d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand)
            for _ in range(n_layers)
        ])
        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

        self.classifier = nn.Linear(d_model, num_classes)
        # self.classifier = nn.Sequential(
        #     nn.Conv1d(128, 256, kernel_size=5, padding=2),
        #     nn.ReLU(),
        #     nn.Flatten(),
        #     nn.Linear(N * 256, num_classes),
        # )

        self.pooling = pooling
        self.mlm_head = nn.Linear(d_model, vocab_size + 1)
        self.mlm_head.weight = self.embedding.weight

    def forward(self, x: torch.LongTensor, mlm=None) -> torch.Tensor:
        h = self.embedding(x)#.to(dtype=torch.get_default_dtype())
        for block in self.layers:
            h = block(h)   # each Mamba block returns [B, L, d_model]
        h = self.norm(h)
        if mlm is not None:
            logits = self.mlm_head(h)  # [B, L, vocab_size + 1]
            return logits
        else:
            # pooled = h[:, 0]
            pooled = h.mean(dim=1)  # [B, d_model]
            pooled = self.dropout(pooled)
            # pooled = pooled.permute(0, 2, 1)
            logits = self.classifier(pooled)  # [B, num_classes]
            return logits


classifier = models.Classifier(
    LENGTH // 4,
    mapping,
    "cuda",
    MambaSequenceClassifier,
    format.to_tetramers,
    True
)

print("Pretraining:")
optimizer = AdamW(classifier.get_parameters(), lr=0.001)
pretraining_data = DataLoader(
    models.SyntheticTetramerDataset(PROCESSED_PRETRAINING_DATA, labels=False),
    batch_size=BATCH_SIZE,
    shuffle=True
)
classifier.pretrain(
    pretraining_data,
    optimizer,
    10_000,
    256,
    patience=3,
    mlm_probability=0.25
)

print("Fine-tuning:")
model = benchmark(
    classifier,
    "MAMBA",
    max_epochs=10,
    loss_fn=nn.CrossEntropyLoss(),
    evaluation_interval=2000,
    learning_rate=0.0001
)
matrix = models.confusion_matrix(model, test_data, "cuda", mapping)
plt.matshow(matrix)
plt.show()
np.set_printoptions(precision=3, suppress=True, linewidth=500, threshold=np.inf)
print(matrix)

Fine-tuning:
Number of parameters: 389_938


 22%|██▏       | 2001/9196 [03:41<9:12:55,  4.61s/it]

P: [0.80326, 0.64982]


 44%|████▎     | 4001/9196 [07:23<6:36:08,  4.58s/it]

P: [0.85748, 0.70958]


...

P: [0.93231, 0.85553]


 44%|████▎     | 4002/9196 [08:11<5:34:05,  3.86s/it]

P: [0.93944, 0.85901]


 50%|█████     | 4622/9196 [09:30<09:24,  8.10it/s]  


KeyboardInterrupt: 

# Frozen Initial Layers

In [ ]:
from torch.optim import Adam, AdamW
import torch
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel, MambaConfig
from mamba_ssm import Mamba
from torch import nn


class MambaBlock(nn.Module):
    def __init__(self, d_model, d_state, d_conv, expand):
        super().__init__()
        self.norm = nn.LayerNorm(d_model)
        self.mamba = Mamba(
            d_model=d_model,
            d_state=d_state,
            d_conv=d_conv,
            expand=expand
        )

    def forward(self, x):
        # Pre-norm + residual connection
        return x + self.mamba(self.norm(x))


class MambaSequenceClassifier(nn.Module):
    def __init__(
        self,
        N: int,
        num_classes: int,
        vocab_size: int = 256,
        d_model: int = 256,
        n_layers: int = 3,
        d_state: int = 16,
        d_conv: int = 4,
        expand: int = 2,
        pooling = "mean",
        dropout: float = 0.1,
    ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size + 1, d_model)
        self.first_layers = nn.ModuleList([
            MambaBlock(d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand)
            for _ in range(n_layers)
        ])
        self.last_layers = nn.ModuleList([
            MambaBlock(d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand)
            for _ in range(n_layers)
        ])
        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

        self.classifier = nn.Linear(d_model, num_classes)
        # self.classifier = nn.Sequential(
        #     nn.Conv1d(128, 256, kernel_size=5, padding=2),
        #     nn.ReLU(),
        #     nn.Flatten(),
        #     nn.Linear(N * 256, num_classes),
        # )

        self.pooling = pooling
        self.mlm_head = nn.Linear(d_model, vocab_size + 1)
        # self.mlm_head.weight = self.embedding.weight

    def forward(self, x: torch.LongTensor, mlm=None) -> torch.Tensor:
        h = self.embedding(x)#.to(dtype=torch.get_default_dtype())
        if mlm is not None:
            for block in self.first_layers:
                h = block(h)   # each Mamba block returns [B, L, d_model]
            for block in self.last_layers:
                h = block(h)   # each Mamba block returns [B, L, d_model]
            h = self.norm(h)
            logits = self.mlm_head(h)  # [B, L, vocab_size + 1]
            return logits
        else:
            for block in self.first_layers:
                h = block(h)   # each Mamba block returns [B, L, d_model]
            for block in self.last_layers:
                h = block(h)   # each Mamba block returns [B, L, d_model]
            h = self.norm(h)
            pooled = h.mean(dim=1)  # [B, d_model]
            pooled = self.dropout(pooled)
            # pooled = pooled.permute(0, 2, 1)
            logits = self.classifier(pooled)  # [B, num_classes]
            return logits

    def freeze_base(self):
        for block in self.first_layers:
            for param in block.parameters():
                param.requires_grad = False

    def unfreeze_base(self):
        for block in self.first_layers:
            for param in block.parameters():
                param.requires_grad = True


classifier = models.Classifier(
    LENGTH // 4,
    mapping,
    "cuda",
    MambaSequenceClassifier,
    format.to_tetramers,
    True
)

print("Pretraining:")
optimizer = AdamW(classifier.get_parameters(), lr=0.001)
pretraining_data = DataLoader(
    models.SyntheticTetramerDataset(PROCESSED_PRETRAINING_DATA, labels=False),
    batch_size=BATCH_SIZE,
    shuffle=True
)
classifier.pretrain(
    pretraining_data,
    optimizer,
    20_000,
    256,
    patience=3,
    mlm_probability=0.1
)
# classifier.model.freeze_base()

print("Fine-tuning:")
model = benchmark(
    classifier,
    "MAMBA",
    max_epochs=10,
    loss_fn=nn.CrossEntropyLoss(),
    evaluation_interval=5000,
    learning_rate=0.0001
)
matrix = models.confusion_matrix(model, test_data, "cuda", mapping)
plt.matshow(matrix)
plt.show()
np.set_printoptions(precision=3, suppress=True, linewidth=500, threshold=np.inf)
print(matrix)


# 128, 3 / 3, 16, 4, 2, mean, 0.1, clip, pretraining: None
# 1/10 T loss: 3.08970. V loss: 1.81286. F1: [0.91764, 0.82083]. P: [0.9162, 0.82467] Patience: 3
# 2/10 T loss: 1.49454. V loss: 1.34284. F1: [0.94197, 0.87123]. P: [0.94196, 0.87661] Patience: 3

# 128, 3 / 3, 16, 4, 2, mean, 0.1, clip, pretraining: 5k, freeze
# 1/10 T loss: 9.88273. V loss: 9.09895. F1: [0.35103, 0.10588]. P: [0.38429, 0.12499]

# 128, 2 / 2, 16, 4, 2, mean, 0.1, clip, pretraining: 10k, freeze
# 1/10 T loss: 3.26994. V loss: 2.42430. F1: [0.89472, 0.75593]. P: [0.89744, 0.76282] Patience: 3

# 128, 2 / 2, 16, 4, 2, mean, 0.1, clip, pretraining: 20k, freeze
# 1/10 T loss: 4.59330. V loss: 3.32864. F1: [0.84704, 0.67053]. P: [0.84721, 0.67978] Patience: 3
# 2/10 T loss: 2.78251. V loss: 2.51102. F1: [0.88506, 0.74861]. P: [0.88034, 0.75483] Patience: 3

# 128, 2 / 2, 16, 4, 2, mean, 0.1, clip, pretraining: 20k, no freeze, span
# 1/10 T loss: 3.30888. V loss: 2.14662. F1: [0.90234, 0.79144]. P: [0.90968, 0.79664] Patience: 3
# 2/10 T loss: 1.76234. V loss: 1.54726. F1: [0.93, 0.84993]. P: [0.92694, 0.85452] Patience: 3
# 3/10 T loss: 1.34409. V loss: 1.35847. F1: [0.93591, 0.86929]. P: [0.93794, 0.87497] Patience: 3

# 128, 2 / 2, 16, 4, 2, mean, 0.1, clip, pretraining: 10k, no freeze, span
# 1/10 T loss: 3.32805. V loss: 1.91426. F1: [0.91257, 0.81601]. P: [0.91242, 0.81589] Patience: 3
# 2/10 T loss: 1.67703. V loss: 1.49529. F1: [0.93111, 0.85618]. P: [0.9346, 0.86045] Patience: 3
# 3/10 T loss: 1.33147. V loss: 1.36263. F1: [0.93998, 0.86843]. P: [0.93879, 0.87145] Patience: 3

# 128, 2 / 2, 16, 4, 2, mean, 0.1, clip, pretraining: 40k, no freeze, span
# 1/10 T loss: 2.66859. V loss: 1.88050. F1: [0.91414, 0.81202]. P: [0.90473, 0.82252] Patience: 3
# 2/10 T loss: 1.59724. V loss: 1.41483. F1: [0.94162, 0.86189]. P: [0.94196, 0.86865] Patience: 3
# 3/10 T loss: 1.28868. V loss: 1.25275. F1: [0.94469, 0.8754]. P: [0.94138, 0.87864] Patience: 3

# 128, 2 / 2, 16, 4, 2, mean, 0.1, clip, pretraining: 20k, no freeze, all masked 15 %
# 1/10 T loss: 2.53448. V loss: 1.84904. F1: [0.91666, 0.8147]. P: [0.90713, 0.82688] Patience: 3
# 2/10 T loss: 1.52909. V loss: 1.40461. F1: [0.94339, 0.86156]. P: [0.9458, 0.86432] Patience: 3
# 3/10 T loss: 1.23919. V loss: 1.23687. F1: [0.95087, 0.8776]. P: [0.95401, 0.87988] Patience: 3

# 128, 3 / 3, 16, 4, 2, mean, 0.1, clip, pretraining: 20k, no freeze, all masked 10 %
# 1/10 T loss: 2.03367. V loss: 1.45920. F1: [0.93885, 0.85342]. P: [0.93636, 0.86178] Patience: 3
# 2/10 T loss: 1.20462. V loss: 1.13109. F1: [0.95273, 0.88658]. P: [0.95174, 0.88691] Patience: 3
# 3/10 T loss: 0.97474. V loss: 1.03495. F1: [0.95888, 0.89874]. P: [0.95798, 0.90307] Patience: 3
# 4/10 T loss: 0.84054. V loss: 0.89632. F1: [0.96534, 0.91211]. P: [0.96508, 0.91451] Patience: 3

Pretraining:


  0%|          | 0/19473 [00:00<?, ?it/s]

Step: 1. Epoch: 1. MLM loss: 5.7056. Patience: 3
Mean entropy: 5.3827


  0%|          | 2/19473 [00:04<10:46:28,  1.99s/it]

Correctly predicted masked tokens: 20 / 4757 (0.42043 %).


...

P: [0.96495, 0.91195]


 27%|██▋       | 2502/9196 [09:44<11:59:09,  6.45s/it]

P: [0.96592, 0.90785]


 29%|██▉       | 2688/9196 [10:22<25:07,  4.32it/s]   


KeyboardInterrupt: 

In [5]:
torch.save(classifier.model.state_dict(), "trained_models/mamba_v1/model.pt2")

# Hierarchical

In [ ]:
BERTAX_TRAIN = BERTAX_DIRECTORY + "train/"
BERTAX_VALIDATION = BERTAX_DIRECTORY + "validation/"
BERTAX_TEST = BERTAX_DIRECTORY + "test/"

from time import time
import json
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch import nn

from stelaro import models
from stelaro.models import autoencoder, feedforward, transformer

LENGTH = 1500
BATCH_SIZE = 64

with open(BERTAX_DIRECTORY + "statistics/map.json", "r") as f:
    mapping = json.load(f)


train_data = DataLoader(
    models.SyntheticMultiLevelTetramerDataset(
        BERTAX_TRAIN,
        mapping,
        (),
        1,
        balance = True,
        other_factor = 0
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)
validation_data = DataLoader(
    models.SyntheticMultiLevelTetramerDataset(
        BERTAX_VALIDATION,
        mapping,
        (),
        1,
        balance = False,
        other_factor = 0
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_data = DataLoader(
    models.SyntheticMultiLevelTetramerDataset(
        BERTAX_TEST,
        mapping,
        (),
        1,
        balance = False,
        other_factor = 0
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)
mapping = train_data.dataset.mapping


def benchmark(
        classifier: models.Classifier,
        name: str,
        max_epochs: int = 20,
        learning_rate: float = 0.001
    ):
    parameters = classifier.get_parameters()
    if parameters:
        optimizer = Adam(classifier.get_parameters(), lr=learning_rate)
        total_params = sum(param.numel() for param in parameters)
        print(f"Number of parameters: {total_params:_}")
    else:
        optimizer = None
    a = time()
    losses, f1, validation_losses = classifier.train(
        train_data,
        validation_data,
        optimizer,
        max_n_epochs=max_epochs,
        patience=3,
        loss_function=nn.CrossEntropyLoss(),
        loss_function_type="supervised",
        evaluation_interval=5000,
    )
    b = time()
    print(f"Training took {(b - a):.3f} s.")
    if losses:
        fig, ax = plt.subplots(1, 2, figsize=(12, 4))
        x = list(range(len(losses)))
        ax[0].plot(x, losses, label="Training")
        ax[0].plot(x, validation_losses, label="Validation")
        ax[0].set(xlabel='Epochs', ylabel='Loss')
        ax[0].set_title("Normalized Loss Against Epochs")
        ax[0].legend()
        ax[1].set(xlabel='Epochs', ylabel="f1")
        ax[1].set_title("F1 Score")
        r = 0
        for f in f1:
            ax[1].plot(x, f, label=f'Rank {r}')
            r += 1
        ax[1].legend()
        fig.suptitle(f"Classification Training for {name}")
        plt.show()
    result = models.evaluate(classifier, test_data, "cuda", mapping)
    rounded_result = [float(f"{r:.5}") for r in result]
    print(f"Test F1 score: {rounded_result}")

    result = models.evaluate_precision(classifier, test_data, "cuda", mapping)
    rounded_result = [float(f"{r:.5}") for r in result]
    print(f"Test precision score: {rounded_result}")

    return classifier